# Exploring Hacker News Posts
Hacker news is a site started by the startup incubator Y Combinator, where user-submitted stories (known as 'posts') are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

The data set can be found [here](https://app.dataquest.io/m/356/guided-project%3A-exploring-hacker-news-posts) here, but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

- `id` - The unique identifier from Hacker News for the post
- `title` - The title of the post
- `url` - The URL that the posts links to, if it the post has a URL
- `num_points` - The number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
- `num_comments` - The number of comments that were made on the post
- `author` - The username of the person who submitted the post
- `created_at` - The date and time at which the post was submitted

We're specifically interested in posts whose titles begin with either `Ask HN` or `Show HN`. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

We'll compare these two types of posts to determine the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?

# Analyzing csv file

In [5]:
# Import file and display few rows
# Remove header
import csv
hn = open('hacker_news.csv')
hn = list(csv.reader(hn))
hn_header = hn[0]
hn = hn[1:]
print(hn_header)
print("\n")
for row in hn[0:5]:
    print(row)
    print("\n")

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']




# Extracting Ask HN and Show HN Posts

In [6]:
ask_posts = []
show_posts = []
other_posts = []
for row in hn:
    title = row[1]
    title = title.lower()
    row[1] = title
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'): 
        show_posts.append(row)
    else:
        other_posts.append(row)


print("Number of 'ask hn' posts = ", len(ask_posts))
print('\n')

print("Number of 'show hn' posts = ", len(show_posts))
print('\n')

print("Number of other posts = ", len(other_posts))
      

Number of 'ask hn' posts =  1744


Number of 'show hn' posts =  1162


Number of other posts =  17194


In [7]:
print(ask_posts[0:2])
print('\n')
print(show_posts[0:2])

[['12296411', 'ask hn: how to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'ask hn: am i the only one outraged by twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43']]


[['10627194', 'show hn: wio link  esp8266 based web of things hardware development platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03'], ['10646440', 'show hn: something pointless i made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46']]


# Calculating the Average Number of Comments for Ask HN and Show HN Posts




In [8]:
total_ask_comments = 0
for post in ask_posts:
    num_comments = int(post[4])
    total_ask_comments += num_comments

avg_ask_comments = total_ask_comments/len(ask_posts)
print('avg_ask_comments=', avg_ask_comments)
    

avg_ask_comments= 14.038417431192661


In [9]:
total_show_comments = 0
for post in show_posts:
    num_comments = int(post[4])
    total_show_comments += num_comments

avg_show_comments = total_show_comments/len(show_posts)
print('avg_show_comments=', avg_show_comments)
    

avg_show_comments= 10.31669535283993


Ask posts receive more comments on average than show posts. We will focus our remaining analysis just on these ask posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

- Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
- Calculate the average number of comments ask posts receive by hour created.

# Finding the Amount of Ask Posts and Comments by Hour Created

In [13]:
import datetime as dt
result_list = [[post[6], int(post[4])] for post in ask_posts]
print(result_list[0:2])   

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29]]


In [20]:
counts_by_hour = {}
comments_by_hour = {}
date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    date = row[0]
    comment = row[1]
    hour = dt.datetime.strptime(date, date_format)
    hour = dt.datetime.strftime(hour, "%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
        
print('hour','number of comments')
for key in comments_by_hour:
    print(key,comments_by_hour[key])
    
print('hour','number of posts')    
for key in counts_by_hour:
    print(key,counts_by_hour[key])      
        

hour number of comments
04 337
17 1146
23 543
12 687
03 421
02 1381
06 397
13 1253
19 1188
22 479
14 1416
11 641
20 1722
15 4477
08 492
10 793
01 683
21 1745
09 251
00 447
18 1439
16 1814
07 267
05 464
hour number of posts
04 47
17 100
23 68
12 73
03 54
02 58
06 44
13 85
19 110
22 71
14 107
11 58
20 80
15 116
08 48
10 59
01 60
21 109
09 45
00 55
18 109
16 108
07 34
05 46


In [28]:
# Arrange the above in descending order
def dict_to_table(table):
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', round(entry[0],2))
        
dict_to_table(comments_by_hour)

15 : 4477
16 : 1814
21 : 1745
20 : 1722
18 : 1439
14 : 1416
02 : 1381
13 : 1253
19 : 1188
17 : 1146
10 : 793
12 : 687
01 : 683
11 : 641
23 : 543
08 : 492
22 : 479
05 : 464
00 : 447
03 : 421
06 : 397
04 : 337
07 : 267
09 : 251


In [22]:
dict_to_table(counts_by_hour)

15 : 116
19 : 110
21 : 109
18 : 109
16 : 108
14 : 107
17 : 100
13 : 85
20 : 80
12 : 73
22 : 71
23 : 68
01 : 60
10 : 59
11 : 58
02 : 58
00 : 55
03 : 54
08 : 48
04 : 47
05 : 46
09 : 45
06 : 44
07 : 34


# Calculating the Average Number of Comments for Ask HN Posts by Hour

In [25]:
avg_by_hour = {}

for hour in comments_by_hour:
    avg_by_hour[hour] = comments_by_hour[hour]/counts_by_hour[hour]

avg_by_hour

{'00': 8.127272727272727,
 '01': 11.383333333333333,
 '02': 23.810344827586206,
 '03': 7.796296296296297,
 '04': 7.170212765957447,
 '05': 10.08695652173913,
 '06': 9.022727272727273,
 '07': 7.852941176470588,
 '08': 10.25,
 '09': 5.5777777777777775,
 '10': 13.440677966101696,
 '11': 11.051724137931034,
 '12': 9.41095890410959,
 '13': 14.741176470588234,
 '14': 13.233644859813085,
 '15': 38.5948275862069,
 '16': 16.796296296296298,
 '17': 11.46,
 '18': 13.20183486238532,
 '19': 10.8,
 '20': 21.525,
 '21': 16.009174311926607,
 '22': 6.746478873239437,
 '23': 7.985294117647059}

In [29]:
dict_to_table(avg_by_hour)

15 : 38.59
02 : 23.81
20 : 21.52
16 : 16.8
21 : 16.01
13 : 14.74
10 : 13.44
14 : 13.23
18 : 13.2
17 : 11.46
01 : 11.38
11 : 11.05
19 : 10.8
08 : 10.25
05 : 10.09
12 : 9.41
06 : 9.02
00 : 8.13
23 : 7.99
07 : 7.85
03 : 7.8
04 : 7.17
22 : 6.75
09 : 5.58


Top 5 Hours for 'Ask HN' Comments

- 15:00: 38.59 average comments per post
- 02:00: 23.81 average comments per post
- 20:00: 21.52 average comments per post
- 16:00: 16.80 average comments per post
- 21:00: 16.01 average comments per post

The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

# Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.

Quick summary:

- We set a goal for the project.
- We collected and sorted the data.
- We reformatted and cleaned the data to prepare it for analysis.
- We analyzed the data.